Reference: 
https://github.com/AlexanderNixon/Machine-learning-reads/blob/master/Movie-content-based-recommender-using-tf-idf.ipynb
https://ithelp.ithome.com.tw/articles/10220479

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

In [3]:
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df = pd.read_csv('./data/courses.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [4]:
id2course_mapping = course_df["course_id"].to_dict()
course2id_mapping = {v : k for k, v in id2course_mapping.items()}
len(course2id_mapping)

728

In [5]:
def clean_data(x):
        return x.replace(" ", "")

fillna=course_df.fillna('')
features= ['course_name', 'teacher_intro', 'sub_groups', 'topics', 'will_learn', 'target_group']
fillna=fillna[features]
for feature in features:
    fillna[feature] = fillna[feature].apply(clean_data)
    
fillna.head(2)

,course_name,teacher_intro,sub_groups,topics,will_learn,target_group
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,熱愛彩妝的人
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"平面設計,繪畫與插畫","Illustrator/以拉,配色技巧",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,每一位興趣的人都能學得來，非常容易的小技巧


In [6]:
def create_combination(x):
    return x['course_name']+ ' ' + x['teacher_intro'] + ' ' + x['sub_groups'] + ' ' + x['topics'] + ' ' + x['will_learn'] + ' ' + x['target_group'] 

fillna['combination'] = fillna.apply(create_combination, axis=1)

In [7]:
fillna.head(2)

,course_name,teacher_intro,sub_groups,topics,will_learn,target_group,combination
0,少女人妻華麗變身：七大妝容七彩的夢幻樂園,在美妝KOL圈裡屬個人風格強烈的Alice，在清新與叛逆風格間遊刃有餘，其幽默的美妝影片手法...,"更多生活品味,護膚保養與化妝","更多生活品味,護膚保養與化妝",不再害怕各種顏色的彩妝，可以更隨心搭配各種繽紛的顏色。\n,熱愛彩妝的人,少女人妻華麗變身：七大妝容七彩的夢幻樂園 在美妝KOL圈裡屬個人風格強烈的Alice，在清新...
1,幾何圖形分割X色塊組合,從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現其實沒有。出社會後從事美...,"平面設計,繪畫與插畫","Illustrator/以拉,配色技巧",可以將任何區塊分割成自己想要的幾何圖形、快速的上色，並且能夠應用在許多地方,每一位興趣的人都能學得來，非常容易的小技巧,幾何圖形分割X色塊組合 從學生時代開始摸索photoshop等軟體，自以為有些天賦但後來發現...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


tfidf = TfidfVectorizer(stop_words='english')
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(fillna['combination'])
# Compute the cosine similarity matrix
cosine_sim2 = linear_kernel(tfidf_matrix, tfidf_matrix)
#Output the shape of tfidf_matrix
tfidf_matrix.shape


# count = CountVectorizer(stop_words='english')
# count_matrix = count.fit_transform(fillna['combination'])

# cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

(728, 20224)

In [16]:
def get_recommendations_new(haved_courses_list, cosine_sim, top = 10):
    cosine_sim_sum = [ [i, 0] for i in range(len(course2id_mapping))]
    haved_courses_index_list = [ course2id_mapping[course_id] for course_id in haved_courses_list]
    for idx in haved_courses_index_list:
        # Get the pairwsie similarity scores of all courses with that course
        sim_scores = list(enumerate(cosine_sim[idx]))
        for i in range(len(sim_scores)):
            cosine_sim_sum[i][1] += sim_scores[i][1]

    # Sort the movies based on the similarity scores
    sim_scores = sorted(cosine_sim_sum, key=lambda x: x[1], reverse=True)

    recommend_indices = []
    # Get the scores of the 10 most course not buy
    for i in range(len(sim_scores)):
        if len(recommend_indices) < top:
            if sim_scores[i][0] not in haved_courses_index_list:
                recommend_indices.append(sim_scores[i][0])

    # Return the top 10 most similar movies
    return course_df['course_id'].iloc[recommend_indices].tolist()
    
get_recommendations_new(['5f194354cad0d086f3ee24cf', '6156a77fdf426a0007cc5fe1'], cosine_sim2, 20)

['5ff6aa28c5cbbcb694532eaa',
 '5e7c50f431e8dd0029db3a65',
 '605ca63e3cd943ce48dc5daa',
 '5fb7565f2945a52e5e2dca8a',
 '6040818e043b47f89a5c8f41',
 '5da7ddea98176600213caf36',
 '60a1cf0ea479bf08e4f5991b',
 '59f1ea744b1842001e21b7c7',
 '5ef0a8a5775dea0566e6cb5f',
 '5ef0a2198e8212c4e6335387',
 '585eb576f108e00800c21f76',
 '5a536d335518aa001e320f7c',
 '578ce3ced0c3030700416c46',
 '589fe5035f188a07007fd5b2',
 '5a6f15aee77921001e624ee9',
 '5ef099ab678184065fd4d426',
 '57a30629d0c303070041dc4b',
 '5c31cb926bbdc8002102405e',
 '5e9c2c813b0256ad483ff17b',
 '5905b47ab684bf0700ec788d']

In [12]:
seen_predict_df = pd.read_csv('./data/test_seen.csv')
predict_user = seen_predict_df["user_id"].to_list()


seen_user_haved_purchased_course = {}
for seen_user_id, course_ids in zip(train_df["user_id"], train_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
for seen_user_id, course_ids in zip(val_seen_df["user_id"], val_seen_df["course_id"]):
    course_id_list = course_ids.split(' ')
    seen_user_haved_purchased_course[seen_user_id] = seen_user_haved_purchased_course.setdefault(seen_user_id, []) + course_id_list
print(len(seen_user_haved_purchased_course))

59737


In [13]:
import csv

predict_users = seen_predict_df["user_id"].to_list()
with open("predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "course_id"])
    for user_id in predict_users:
        recommend = " ".join(get_recommendations_new(seen_user_haved_purchased_course[user_id], cosine_sim2, top = 50))
        writer.writerow([user_id, recommend])